# 2.1 - Data Cleaning

## Importing the libraries

In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import seaborn as sns

We can refer to the Pandas module using the "dot notation" to call its methods. To read our data (which is in the form of csv files), into a so-called DataFrame structure, we use the method read_csv() and pass in each file name as a string: All the datasets are loaded and storeed in pandas dataframe objects.

## Importing the datasets

In [ ]:
%%time
info_content_raw_df = pd.read_csv('../data/raw/Info_Content.csv', index_col='ucid')
info_userdata_raw_df = pd.read_csv('../data/raw/Info_UserData.csv', index_col='uuid')
log_problem_raw_df = pd.read_csv('../data/raw/Log_Problem.csv', index_col='upid')

In [ ]:
# scale down some records
# log_problem_raw_df = log_problem_raw_df[log_problem_raw_df['timestamp_TW'] < '2019-01-01']

In [ ]:
log_problem_raw_df.shape

## Drop unused columns

In [ ]:
selected_columns = ['content_pretty_name','difficulty','learning_stage']

In [ ]:
info_content_df = info_content_raw_df[selected_columns].copy()

In [ ]:
info_content_df.head()

In [ ]:
info_userdata_df = info_userdata_raw_df.copy()

In [ ]:
info_userdata_df.head()

In [ ]:
selected_columns = ['gender','points','badges_cnt','first_login_date_TW','user_grade','user_city','has_teacher_cnt','is_self_coach','has_student_cnt','belongs_to_class_cnt','has_class_cnt']

In [ ]:
log_problem_df = log_problem_raw_df.copy()

In [ ]:
log_problem_df.head()

## Handling missing values

The first thing to do when you get a new dataset is take a look at some of it. This lets you see that it all read in correctly and gives an idea of what's going on with the data. In this case, let's see if there are any missing values, which will be reprsented with NaN or None.

In [ ]:
# get the number of missing data points per column
missing_values_count = info_content_df.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count

In [ ]:
# how many total missing values do we have?
total_cells = np.product(info_content_df.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

In [ ]:
# get the number of missing data points per column
missing_values_count = info_userdata_df.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count

In [ ]:
# how many total missing values do we have?
total_cells = np.product(info_userdata_df.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

In [ ]:
# replace all NA's with unspecified
info_userdata_df = info_userdata_df.fillna('unspecified')

In [ ]:
# get the number of missing data points per column
missing_values_count = log_problem_df.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count

In [ ]:
# how many total missing values do we have?
total_cells = np.product(log_problem_df.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

In [ ]:
# replace all NA's with false
log_problem_df = log_problem_df.fillna(False)

## Parsing Date

Now that we know that our date column isn't being recognized as a date, it's time to convert it so that it is recognized as a date. This is called "parsing dates" because we're taking in a string and identifying its component parts.

In [ ]:
info_userdata_df['first_login_date_TW'].head()

In [ ]:
log_problem_df['timestamp_TW'].head()

### Convert our date columns to datetime

We can pandas what the format of our dates are with a guide called as "strftime directive", which you can find more information on at this link. The basic idea is that you need to point out which parts of the date are where and what punctuation is between them. There are lots of possible parts of a date, but the most common are %d for day, %m for month, %y for a two-digit year and %Y for a four digit year.

In [ ]:
info_userdata_df['first_login_date_TW'] = pd.to_datetime(info_userdata_df['first_login_date_TW'], format='%Y-%m-%d')
info_userdata_df = info_userdata_df.rename(columns = {'first_login_date_TW':'date_login'})

In [ ]:
log_problem_df['timestamp_TW'] = pd.to_datetime(log_problem_df['timestamp_TW'], format='%Y-%m-%d %H:%M:%S %Z')
log_problem_df = log_problem_df.rename(columns = {'timestamp_TW':'datetime_attempt'})

Now when I check the first few rows of the new column, I can see that the dtype is `datetime64`. I can also see that my dates have been slightly rearranged so that they fit the default order datetime objects (year-month-day).

In [ ]:
# print the first few rows
info_userdata_df['date_login'].head()

In [ ]:
log_problem_df['datetime_attempt'].head()

### Select Day of Month

In [ ]:
# get the day of the month from the date_parsed column
day_of_month_userdata = info_userdata_df['date_login'].dt.day
day_of_month_userdata.head()

In [ ]:
# remove na's
day_of_month_userdata = day_of_month_userdata.dropna()

# plot the day of the month
sns.distplot(day_of_month_userdata, kde=False, bins=31)

In [ ]:
# get the day of the month from the date_parsed column
day_of_month_problem = log_problem_df['datetime_attempt'].dt.day
day_of_month_problem.head()

In [ ]:
# remove na's
day_of_month_problem = day_of_month_problem.dropna()

# plot the day of the month
sns.distplot(day_of_month_problem, kde=False, bins=31)

Yep, it looks like we did parse our dates correctly & this graph makes good sense to me.

## Exporting Data

The data is in DataFrame form, it can manipulate it by adding a body mass index column. The cleaned data is exported to the interim data folder which stores the intermediate data that has been transformed.

In [ ]:
info_content_df.to_csv('../data/interim/Info_Content.csv')
info_userdata_df.to_csv('../data/interim/Info_UserData.csv')
log_problem_df.to_csv('../data/interim/Log_Problem.csv')